In [1]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ["TRANSFORMERS_CACHE"]="/workspace/cache"
os.environ["HF_DATASETS_CACHE"]="/workspace/cache" 

import pandas as pd
import numpy as np

import datasets
from datasets import load_dataset
import datasets
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader
import torch

import os
import requests
from tqdm import tqdm
import pickle 
from PIL import Image
from PIL import ImageFile                                                      
ImageFile.LOAD_TRUNCATED_IMAGES = True

from glob import glob
import time

/root/anaconda3/envs/llava/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
dset = datasets.load_from_disk("/workspace/sea_vl/ALL_DATA_filtered")

In [3]:
model = SentenceTransformer("sentence-transformers/clip-ViT-B-32").to('cuda')

/root/anaconda3/envs/llava/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
sea_vqa_dataset = load_dataset('wit543/sea-vqa')

In [4]:
cvqa_dataset = load_dataset('afaji/cvqa')

cvqa_sea_subsets = [
    "('Indonesian', 'Indonesia')",
    "('Malay', 'Malaysia')",
    "('Javanese', 'Indonesia')",
    "('Minangkabau', 'Indonesia')",
    "('Sundanese', 'Indonesia')",
    "('Chinese', 'Singapore')"
]
cvqa_dataset_filt = cvqa_dataset['test'].filter(lambda x: str(x['Subset']) in cvqa_sea_subsets, num_proc=32)

Filter (num_proc=32):   0%|          | 0/10374 [00:00<?, ? examples/s]

# Extract SEA-VQA

In [7]:
sea_vqa_images_filt = []
sea_vqa_images_embed = []
sea_vqa_caption = []
sea_vqa_culture = []
for key in sea_vqa_dataset.keys():
    for row in tqdm(sea_vqa_dataset[key]):
        try:
            img_opened = Image.open(requests.get(row['image_path'], stream=True).raw)
            sea_vqa_images_embed.append(model.encode(img_opened))
            sea_vqa_images_filt.append(img_opened)
            if row['correct_answer'] in ['a', 'b', 'c', 'd']:
                sea_vqa_caption.append(row['question'] + " " + row['choice_' + row['correct_answer']])
            else:
                sea_vqa_caption.append(row['question'])
            sea_vqa_culture.append(key)
        except:
            print(row)
pickle.dump((sea_vqa_images_filt, sea_vqa_images_embed, sea_vqa_caption, sea_vqa_culture), open('sea_vqa.pkl', 'wb'))

100%|████████████████████████████████████████████████████████████████████████| 313/313 [09:07<00:00,  1.75s/it]


In [8]:
(sea_vqa_images_filt, sea_vqa_images_embed, sea_vqa_caption, sea_vqa_culture) = pickle.load(open('sea_vqa.pkl', 'rb'))

In [9]:
len(sea_vqa_images_filt), len(sea_vqa_images_embed), len(sea_vqa_caption), len(sea_vqa_culture)

(1999, 1999, 1999, 1999)

# Extract CVQA

In [10]:
cvqa_images_filt = []
cvqa_images_embed = []
cvqa_caption = []
cvqa_culture = []
for row in tqdm(cvqa_dataset_filt):
    try:
        cvqa_images_embed.append(model.encode(row['image']))
        cvqa_images_filt.append(row['image'])
        cvqa_caption.append(row['Translated Question'] + " " + ', '.join(row['Translated Options']))
        cvqa_culture.append(eval(row['Subset'])[0])
    except:
        print(row)
pickle.dump((cvqa_images_filt, cvqa_images_embed, cvqa_caption, cvqa_culture), open('cvqa.pkl', 'wb'))

100%|██████████████████████████████████████████████████████████████████████| 1687/1687 [04:07<00:00,  6.83it/s]


In [11]:
pickle.dump((cvqa_images_filt, cvqa_images_embed, cvqa_caption, cvqa_culture), open('cvqa.pkl', 'wb'))

In [12]:
cvqa_images_embed = []
cvqa_caption = []
cvqa_culture = []
cvqa_category = []
for row in tqdm(cvqa_dataset['test']):
    try:
        cvqa_images_embed.append(model.encode(row['image']))
        cvqa_caption.append(row['Translated Question'] + " " + ', '.join(row['Translated Options']))
        cvqa_culture.append(eval(row['Subset'])[0])
        cvqa_category.append(row['Category'])
    except:
        print(row)
pickle.dump((cvqa_images_embed, cvqa_caption, cvqa_culture, cvqa_category), open('cvqa_category_all.pkl', 'wb'))

100%|████████████████████████████████████████████████████████████████████| 10374/10374 [21:25<00:00,  8.07it/s]


# Extract Coyo

In [ ]:
bs = 64
coyo_images_embed = []
coyo_images_filt = []
coyo_caption = []

def invalid_images_as_none(batch):
    images = []
    for image_url in batch["url"]:
        try:
            image = Image.open(requests.get(image_url, stream=True, timeout=8).raw).convert('RGB')
        except Exception:
            image = None
        images.append(image)
    batch["image"] = images
    return batch

dset = datasets.load_from_disk("/workspace/sea_vl/ALL_DATA_filtered")
dset = dset.with_transform(invalid_images_as_none)

loader = DataLoader(dset, batch_size=bs, num_workers=bs, prefetch_factor=8, collate_fn=lambda x: {k: [row[k] for row in x] for k in x[0]})
for i, batch in tqdm(enumerate(loader)):
    imgs = []
    for i, img in enumerate(batch['image']):
        if img is not None:
            if img.size[0] < 50 or img.size[1] < 50:
                continue
            imgs.append(img)
            coyo_images_filt.append(batch['url'][i])
            coyo_caption.append(batch['text'][i])
    
    img_embeds = model.encode(imgs, batch_size=bs)
    for img_emb in img_embeds:
        coyo_images_embed.append(img_emb)

    if i == len(loader) - 1:
        break

0it [00:00, ?it/s]/root/anaconda3/envs/llava/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/root/anaconda3/envs/llava/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/root/anaconda3/envs/llava/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/root/anaconda3/envs/llava/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/root/anaconda3/envs/llava/lib/python3.10/site-packages/PIL/Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/root/anaconda3/envs/l

In [ ]:
print(len(coyo_images_embed), len(coyo_images_filt), len(coyo_caption), flush=True)
pickle.dump((coyo_images_filt, coyo_images_embed, coyo_caption, []), open('./coyo.pkl', 'wb'))